<a href="https://colab.research.google.com/github/AleX5andr/ICH_Final_project/blob/main/notebooks/02_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries


In [ ]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import pickle

pd.set_option('display.max_columns', None)

# Variables

In [ ]:
non = "unknown"

#Loading data

In [ ]:
url = ("https://drive.google.com/file/d/1dXIHcg6AvTe4xSPg9CnndlQo3xzuFzRY/"
                                                            "view?usp=sharing")
url = url.split("/d/")[1].split("/")[0]
url = f"https://drive.google.com/uc?export=download&id={url}"
datasets = pd.read_pickle(url)

calls = datasets['calls']
contacts = datasets['contacts']
deals = datasets['deals']
spend = datasets['spend']

# Time series analysis

## Analyzing deal creation trends over time and their relationship to calls

In [ ]:
deals['Created Date'] = deals['Created Time'].dt.date
calls['Call Date'] = calls['Call Start Time'].dt.date
deals_per_day = (deals.groupby('Created Date').size()
                                            .reset_index(name='Deals Count'))
calls_per_day = (calls.groupby('Call Date').size()
                                            .reset_index(name='Calls Count'))
time_series_df = (pd.merge(deals_per_day, calls_per_day, left_on='Created Date',
                                    right_on='Call Date', how='outer').dropna())
time_series_df.rename(columns={'Created Date': 'Date'}, inplace=True)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=time_series_df['Date'],
    y=time_series_df['Deals Count'],
    mode='lines',
    name='Deals Count',
    line=dict(color='cornflowerblue', width=2)
))
fig.add_trace(go.Scatter(
    x=time_series_df['Date'],
    y=time_series_df['Calls Count'],
    mode='lines',
    name='Calls Count',
    line=dict(color='orange', width=2)
))
fig.update_layout(
    title='Trends of Deals and Calls Over Time',
    xaxis_title='Date',
    yaxis_title='Count',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.5)'),
    template='plotly_white',
    xaxis_tickangle=-45,
    hovermode='x unified',
    height=500
)
fig.show()

In [ ]:
fig = go.Figure()
time_series_df['Date'] = pd.to_datetime(time_series_df['Date'])
fig.add_trace(go.Scatter(x=time_series_df['Date'],
            y=time_series_df['Deals Count'], mode='lines', name='Deals Count'))
deals_trend = px.scatter(time_series_df, x='Date', y='Deals Count',
                                    trendline='ols', template='plotly_white')
trend_data = deals_trend.data[1]
fig.add_trace(trend_data)
fig.add_trace(go.Scatter(x=time_series_df['Date'],
            y=time_series_df['Calls Count'], mode='lines', name='Calls Count'))
calls_trend = (px.scatter(time_series_df, x='Date', y='Calls Count',
                                    trendline='ols', template='plotly_white'))
trend_data = calls_trend.data[1]
fig.add_trace(trend_data)
fig.update_layout(
    title='Trends of Deals and Calls Over Time with Trend Lines',
    xaxis_title='Date',
    yaxis_title='Count',
    legend_title='Legend',
    xaxis=dict(tickangle=45),
    template='plotly_white'
)
fig.show()

The number of calls and transactions overall demonstrates an upward trend:
*   the number of calls is growing steadily, with significant fluctuations (likely depending on the day of the week or advertising campaigns)
*   the number of transactions is also increasing, but at a slower rate—the growth is gradual and lags behind the number of callsт

This indicates a positive relationship between call activity and the number of transactions: an increase in the number of customer contacts is accompanied by a gradual increase in the number of completed transactions.

In [ ]:
numeric_time_series_df = time_series_df[['Date', 'Deals Count', 'Calls Count']]
weekly_time_series_df = (numeric_time_series_df.resample('W', on='Date').sum()
                                                                .reset_index())

fig = go.Figure()
fig.add_trace(go.Scatter(x=weekly_time_series_df['Date'],
                y=weekly_time_series_df['Deals Count'], mode='lines+markers',
                name='Deals Count (Weekly)'))
deals_trend = px.scatter(weekly_time_series_df, x='Date', y='Deals Count',
                         trendline='ols', template='plotly_white')
trend_data = deals_trend.data[1]
fig.add_trace(trend_data)
fig.add_trace(go.Scatter(x=weekly_time_series_df['Date'],
                         y=weekly_time_series_df['Calls Count'],
                         mode='lines+markers', name='Calls Count (Weekly)'))
calls_trend = px.scatter(weekly_time_series_df, x='Date', y='Calls Count',
                         trendline='ols', template='plotly_white')
trend_data = calls_trend.data[1]
fig.add_trace(trend_data)
fig.update_layout(
    title='Weekly Trends of Deals and Calls Over Time with Trend Lines',
    xaxis_title='Date',
    yaxis_title='Count',
    legend_title='Legend',
    xaxis=dict(tickangle=45),
    template='plotly_white'
)
fig.show()

In [ ]:
corr = time_series_df['Calls Count'].corr(time_series_df['Deals Count'])
x = time_series_df['Calls Count']
y = time_series_df['Deals Count']
coeffs = np.polyfit(x, y, 1)
trendline = np.polyval(coeffs, x)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode='markers',
    name='Data Points',
    marker=dict(size=6, color='blue', opacity=0.6)
))
fig.add_trace(go.Scatter(
    x=x,
    y=trendline,
    mode='lines',
    name='Trend Line',
    line=dict(color='red', width=2)
))
fig.update_layout(
        title=f"Relationship between Calls and Deals "
                                                "orrelation (r) = {corr:.2f}",
    xaxis_title="Calls per Day",
    yaxis_title="Deals per Day",
    template="plotly_white",
    width=800,
    height=500
)
fig.show()

The diagram shows a moderately positive correlation between the number of calls and the number of deals (r = 0.57).
This means that as the number of calls increases, the number of deals also increases, although the relationship is not perfectly linear—other factors (such as call quality or deal stage) also influence the result.

## Distribution of trade closing times and duration of the period from creation to closing

In [ ]:
df = deals.copy()
total_deals = deals.shape[0]
total_deals = deals.shape[0]
closed = deals[deals['Closing Date'].notna()].copy()
closed_share = round(len(closed) / total_deals * 100, 2)
print(f"Closed deals: {len(closed)} out of {total_deals} ({closed_share}%)\n")

df['Closing Date'] = pd.to_datetime(df['Closing Date'], errors='coerce')
df['Created Date'] = pd.to_datetime(df['Created Time'], errors='coerce')
closed = df[df['Closing Date'].notna()].copy()
closed['duration_days'] = ((closed['Closing Date'] - closed['Created Date'])
                                                                .dt.days + 1)
duration_desc = (closed['duration_days']
    .describe(percentiles=[.25, .5, .75, .9, .95, .99]).round(1).to_frame().T)
print("Descriptive statistics for deal duration (in days):")
display(duration_desc)

fig = go.Figure()
fig.add_trace(go.Histogram(
    x=closed['duration_days'],
    nbinsx=40,
    name='Deal Duration',
    marker=dict(color='royalblue', opacity=0.7),
    hovertemplate='Duration: %{x:.0f} days<br>Deals: %{y}'
))
fig.update_layout(
    title='Distribution of Deal Duration (days)',
    xaxis_title='Duration (days)',
    yaxis_title='Number of Deals',
    bargap=0.05,
    template='plotly_white',
    width=800,
    height=400
)
fig.show()

bins = [-1, 0, 7, 30, 90, np.inf]
labels = ['Same day', '≤1 week', '1–4 weeks', '1–3 months', '>3 months']
closed['duration_group'] = (pd.cut(closed['duration_days'], bins=bins,
                                                                labels=labels))
duration_stats = (
    closed['duration_group']
    .value_counts()
    .reindex(labels)
    .to_frame(name='Count')
)
duration_stats['Share'] = (
    (duration_stats['Count'] / duration_stats['Count'].sum() * 100)
    .round(1)
    .astype(str) + '%'
)
print("\nDistribution of deals by closing duration:")
display(duration_stats)

Closed deals: 13061 out of 19667 (66.41%)

Descriptive statistics for deal duration (in days):


,count,mean,std,min,25%,50%,75%,90%,95%,99%,max
duration_days,13061.0,16.2,32.6,0.0,1.0,4.0,13.0,47.0,84.0,167.0,358.0



Distribution of deals by closing duration:


,Count,Share
duration_group,,
Same day,2344,17.9%
≤1 week,6072,46.5%
1–4 weeks,2766,21.2%
1–3 months,1306,10.0%
>3 months,573,4.4%


Of the 19,667 deals, 66.4% were closed.

The average deal cycle length is approximately 16 days, with a median of 4 days, indicating a significant right-hand skew (many deals close quickly, but some take up to a year).

By group:
*   46.5% of deals close within the first week
*   another 21.2% close within 1-4 weeks
*   approximately 14.4% take more than a month
*   only 4.4% take more than 3 months

Most deals close within the first week, but a long tail indicates the presence of slow-moving cases that may require individual analysis.

# Campaign performance analysis

## The effectiveness of different campaigns in terms of lead generation and conversion rate

In [ ]:
df = deals.copy()
df = df[df['Campaign'].notna() & (df['Campaign'] != non)]
campaign_stats = (
    df.groupby('Campaign', dropna=False)
    .agg(
        leads=('Id', 'count'),
        successful=('Stage', lambda x: (x == 'Payment Done').sum())
    )
    .reset_index()
)
campaign_stats['conversion_rate'] = ((campaign_stats['successful'] /
                                      campaign_stats['leads'] * 100).round(2))
campaign_stats = (campaign_stats.sort_values('leads', ascending=False)
                                                                .reset_index())
campaign_stats['conversion_rate'] = (campaign_stats['conversion_rate']
                                                        .astype("str") + "%")
print("Campaign effectiveness (by leads and conversion rate):")
display(campaign_stats.head(20))

Campaign effectiveness (by leads and conversion rate):


,index,Campaign,leads,successful,conversion_rate
0,117,performancemax_digitalmarkt_ru_DE,2573,112,4.35%
1,150,youtube_shorts_DE,1596,53,3.32%
2,17,12.07.2023wide_DE,1531,48,3.14%
3,1,02.07.23wide_DE,943,52,5.51%
4,4,04.07.23recentlymoved_DE,734,31,4.22%
5,3,03.07.23women,595,31,5.21%
6,59,Dis_DE,567,30,5.29%
7,7,07.07.23LAL_DE,529,28,5.29%
8,18,12.09.23interests_Uxui_DE,515,27,5.24%
9,39,24.09.23retargeting_DE,472,17,3.6%


"performancemax_digitalmarkt_ru_DE" (2,573 leads) and "youtube_shorts_DE" (1,596 leads) generate the most leads, but their conversion rates are relatively low—around 3–4%.

The "02.07.23wide_DE, 07.07.23LAL_DE, Dis_DE," and "03.07.23women" campaigns show more balanced results—with a moderate lead volume (500–900), they achieve a conversion rate of around 5%, indicating a good target audience.

The best conversion rate is demonstrated by "brand_search_eng_DE" (9.3%), indicating high-quality traffic. At the same time, webinars ("webinar1604", "webinar1906") and some low-reach campaigns ("bbo_DE") have low or no conversion rates, which may indicate ineffective creatives or an inappropriate audience.

Overall, it's worth focusing on optimizing large campaigns with low conversion rates and scaling up search and targeted campaigns with high success rates.

## The effectiveness of various marketing sources "Source" in generating high-quality leads

In [ ]:
df = deals.copy()
if df['Source'].dtype.name == 'category':
    if non not in df['Source'].cat.categories:
        df['Source'] = df['Source'].cat.add_categories([non])
df['Source'] = df['Source'].fillna(non)
source_stats = (
    df.groupby('Source', dropna=False, observed=True)
    .agg(
        total_leads=('Id', 'count'),
        qualified_leads=('Quality', lambda x: x.isin(['A - High', 'B - Medium',
                                                      'C - Low']).sum()),
        successful=('Stage', lambda x: (x == 'Payment Done').sum())
    )
    .reset_index()
)
source_stats['% Qualified'] = ((source_stats['qualified_leads'] /
                                source_stats['total_leads'] * 100).round(1))
source_stats['% Successful'] = ((source_stats['successful'] /
                                source_stats['total_leads'] * 100).round(1))
numeric_cols = source_stats.select_dtypes(include=['number']).columns
source_stats[numeric_cols] = source_stats[numeric_cols].fillna(0)
source_stats['Qualified'] = (
    source_stats['qualified_leads'].astype(int).astype(str) +
    " (" + source_stats['% Qualified'].astype(str) + "%)"
)
source_stats['Successful'] = (
    source_stats['successful'].astype(int).astype(str) +
    " (" + source_stats['% Successful'].astype(str) + "%)"
)
source_stats = (source_stats[['Source', 'total_leads', 'Qualified',
                              'Successful']])
source_stats = source_stats.sort_values('total_leads', ascending=False)
print("Marketing Source Effectiveness:")
display(source_stats)

Marketing Source Effectiveness:


,Source,total_leads,Qualified,Successful
2,Facebook Ads,4730,1457 (30.8%),202 (4.3%)
3,Google Ads,4114,1087 (26.4%),172 (4.2%)
9,Tiktok Ads,2003,473 (23.6%),56 (2.8%)
7,SMM,1669,502 (30.1%),91 (5.5%)
11,Youtube Ads,1618,427 (26.4%),53 (3.3%)
5,Organic,1498,566 (37.8%),146 (9.7%)
1,CRM,1456,125 (8.6%),24 (1.6%)
0,Bloggers,1074,300 (27.9%),39 (3.6%)
8,Telegram posts,993,294 (29.6%),40 (4.0%)
10,Webinar,306,96 (31.4%),26 (8.5%)


Facebook Ads and Google Ads generate the largest number of leads, but Organic and Webinar generate a higher proportion of qualified leads.

Google Ads, however, demonstrates a balanced balance—high volume and a moderately high percentage of qualified leads—making it a key channel for scaling.

# Sales department performance analysis

Evaluate the performance of individual deal owners and campaigns in terms of number of deals processed, conversion rate, and total sales amount

## Sales Manager Effectiveness (Deal Owner Name)

In [ ]:
df = deals.copy()
df['is_closed'] = df['Closing Date'].notna()
df['is_success'] = df['Stage'].eq('Payment Done')
df['has_payment'] = df['Offer Total Amount'].fillna(0) > 0
sales_by_owner = (
    df.groupby('Deal Owner Name', observed=True)
    .agg(
        total_deals=('Id', 'count'),
        closed_deals=('is_closed', 'sum'),
        successful_deals=('is_success', 'sum'),
        total_sales=('Offer Total Amount', 'sum'),
        avg_sale=('Offer Total Amount', 'mean')
    )
    .reset_index()
)
sales_by_owner['closed_%'] = ((sales_by_owner['closed_deals'] /
                        sales_by_owner['total_deals'] * 100).round(1))
sales_by_owner['success_%'] = ((sales_by_owner['successful_deals'] /
                        sales_by_owner['total_deals'] * 100).round(1))
sales_by_owner['Closed Deals'] = (sales_by_owner.apply(lambda x:
                        f"{int(x['closed_deals'])} ({x['closed_%']}%)", axis=1))
sales_by_owner['Successful Deals'] = (sales_by_owner.apply(lambda x:
                f"{int(x['successful_deals'])} ({x['success_%']}%)", axis=1))
sales_by_owner['conversion_rate'] = (
    (sales_by_owner['successful_deals'] / sales_by_owner['total_deals'] * 100)
    .round(1).astype(str) + '%'
)
sales_by_owner['total_sales'] = (sales_by_owner['total_sales'].fillna(0)
                                                        .round(0).astype(int))
sales_by_owner['avg_sale'] = (sales_by_owner['avg_sale'].fillna(0)
                                                        .round(0).astype(int))
sales_by_owner = sales_by_owner[
    [
        'Deal Owner Name', 'total_deals',
        'Closed Deals', 'Successful Deals',
        'total_sales', 'avg_sale', 'conversion_rate'
    ]
].sort_values(by='total_sales', ascending=False)
print("Sales effectiveness by Deal Owner:")
display(sales_by_owner)

Sales effectiveness by Deal Owner:


,Deal Owner Name,total_deals,Closed Deals,Successful Deals,total_sales,avg_sale,conversion_rate
21,Ulysses Adams,2057,1397 (67.9%),140 (6.8%),4956300,8247,6.8%
5,Charlie Davis,2791,2016 (72.2%),148 (5.3%),3787500,6622,5.3%
12,Julia Nelson,2063,1378 (66.8%),92 (4.5%),3512810,8695,4.5%
17,Paula Underwood,1749,1203 (68.8%),91 (5.2%),2765000,8560,5.2%
22,Victor Barnes,1182,436 (36.9%),44 (3.7%),2144800,8683,3.7%
2,Ben Hall,1287,537 (41.7%),46 (3.6%),2030800,7902,3.6%
15,Nina Scott,1214,823 (67.8%),46 (3.8%),1914400,8822,3.8%
18,Quincy Vincent,1782,1271 (71.3%),65 (3.6%),1788900,8856,3.6%
16,Oliver Taylor,160,64 (40.0%),50 (31.2%),1626500,10843,31.2%
10,Jane Smith,904,827 (91.5%),31 (3.4%),1411000,4968,3.4%


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=sales_by_owner['Deal Owner Name'],
    y=sales_by_owner['total_sales'],
    name='Total Sales (€)',
    marker_color='royalblue'
))
fig.add_trace(go.Scatter(
    x=sales_by_owner['Deal Owner Name'],
    y=sales_by_owner['conversion_rate'],
    name='Conversion Rate (%)',
    yaxis='y2',
    mode='lines+markers',
    line=dict(color='orange', width=3)
))
fig.update_layout(
    title="Sales Department Efficiency by Deal Owner",
    xaxis_title="Deal Owner",
    yaxis=dict(title="Total Sales (€)"),
    yaxis2=dict(title="Conversion Rate (%)", overlaying='y', side='right'),
    template="plotly_white",
    height=500
)
fig.show()

The most productive managers are Ulysses Adams, Charlie Davis, and Julia Nelson—they handle the largest number of deals and demonstrate a consistent conversion rate of 4-7%.

Oliver Taylor, however, stands out with a record-breaking deal success rate (31%), despite a lower lead volume, indicating high client effectiveness.

Some employees (Bob Brown, Rachel White, Amy Green) close deals but do not result in successful payments, which may indicate issues with the conversion to payment stage.

## Advertising Campaign Effectiveness

In [ ]:
campaign_eff = (
    df.groupby('Campaign', observed=True).agg(
        total_deals=('Id', 'count'),
        successful_deals=('is_success', 'sum'),
        total_sales=('Offer Total Amount', 'sum')
    ).reset_index())
campaign_eff['total_sales'] = campaign_eff['total_sales'].fillna(0).astype(int)
campaign_eff['conversion_rate'] = (
    (campaign_eff['successful_deals'] / campaign_eff['total_deals'] * 100)
    .round(1).astype(str) + '%')
campaign_eff = campaign_eff.sort_values('total_sales', ascending=False)
print("Top 15 Campaign Effectiveness:")
display(campaign_eff[:15])

Top 15 Campaign Effectiveness:


,Campaign,total_deals,successful_deals,total_sales,conversion_rate
117,performancemax_digitalmarkt_ru_DE,2573,112,3299100,4.4%
17,12.07.2023wide_DE,1531,48,2053400,3.1%
150,youtube_shorts_DE,1596,53,1984600,3.3%
1,02.07.23wide_DE,943,52,1661100,5.5%
59,Dis_DE,567,30,1228800,5.3%
18,12.09.23interests_Uxui_DE,515,27,1221300,5.2%
3,03.07.23women,595,31,1140698,5.2%
4,04.07.23recentlymoved_DE,734,31,1124100,4.2%
7,07.07.23LAL_DE,529,28,1033500,5.3%
39,24.09.23retargeting_DE,472,17,980400,3.6%


An analysis of the top 15 marketing campaigns shows that:
*   "performancemax_digitalmarkt_ru_DE" leads in sales volume (≈€3.3 million) and number of successful deals (112), despite a moderate conversion rate of 4.4%
*   the campaigns "12.07.2023wide_DE," "youtube_shorts_DE," and "02.07.23wide_DE" also demonstrate a high contribution to sales (≈€1.6–2 million) with a conversion rate of 3–5%
*   the most effective campaign in terms of conversion rate is "brand_search_eng_DE" at 9.3%, although it is inferior to larger performance campaigns in terms of deal volume
*   most of the top campaigns maintain conversion rates in the 4–6% range, indicating stable, but not maximal, advertising funnel effectiveness

A combination of campaigns (high lead volume) and niche search (high conversion) provides the most balanced results in terms of sales volume and lead quality.

# Payment and product analysis

## Distribution of payment types and their impact on transaction success

In [ ]:
df = deals.copy()
df['is_success'] = df['Stage'].eq('Payment Done')
payment_stats = (
    df.groupby('Payment Type', observed=True)
    .agg(
        total_deals=('Id', 'count'),
        successful=('is_success', 'sum')
    )
    .reset_index()
)
payment_stats['success_rate'] = ((payment_stats['successful'] /
                                  payment_stats['total_deals'] * 100).round(1))
payment_stats['Successful Deals'] = payment_stats.apply(
    lambda x: f"{int(x['successful'])} ({x['success_rate']}%)", axis=1
)
payment_stats = (payment_stats[['Payment Type', 'total_deals',
                                                        'Successful Deals']])
payment_stats = payment_stats.sort_values('total_deals', ascending=False)
print("Payment Type Effectiveness:")
display(payment_stats)

Payment Type Effectiveness:


,Payment Type,total_deals,Successful Deals
1,Recurring Payments,340,248 (72.9%)
0,One Payment,137,113 (82.5%)
2,Reservation,5,1 (20.0%)


*   Recurring Payments account for the majority of transactions—340 transactions—with a success rate of 72.9%, indicating a stable and reliable payment model for most clients.
*   One Payment (one-time payment) demonstrates the highest success rate—82.5%, although there are fewer such transactions, indicating a simpler and faster conversion with full payment.
*   Reservation is used extremely rarely (5 transactions) and has a low success rate (20%), likely due to post-booking cancellations.

Thus, both main models (especially One Payment) demonstrate high efficiency, while Reservation can be considered the least effective payment type.

## Analyze the popularity and success of different products and types of training

In [ ]:
df = deals.copy()
df['is_success'] = df['Stage'].eq('Payment Done')
product_stats = (
    df.groupby('Product', observed=True).agg(
        total_deals=('Id', 'count'),
        successful=('is_success', 'sum')
    ).reset_index())
product_stats['conversion_rate'] = (
    product_stats['successful'] / product_stats['total_deals'] * 100
).round(1)
edu_stats = (
    df.groupby('Education Type', observed=True).agg(
        total_deals=('Id', 'count'),
        successful=('is_success', 'sum')
    ).reset_index())
edu_stats['conversion_rate'] = (
    edu_stats['successful'] / edu_stats['total_deals'] * 100).round(1)

print("Product Effectiveness:")
product_stats['conversion_rate'] = (product_stats['conversion_rate']
                                                            .astype(str) + "%")
display(product_stats.sort_values('total_deals', ascending=False))
print("\nEducation Type Effectiveness:")
edu_stats['conversion_rate'] = edu_stats['conversion_rate'].astype(str) + "%"
display(edu_stats.sort_values('total_deals', ascending=False))

Product Effectiveness:


,Product,total_deals,successful,conversion_rate
1,Digital Marketing,1944,472,24.3%
3,UX/UI Design,1004,227,22.6%
4,Web Developer,565,136,24.1%
2,Find yourself in IT,3,1,33.3%
0,Data Analytics,1,0,0.0%



Education Type Effectiveness:


,Education Type,total_deals,successful,conversion_rate
1,Morning,2837,658,23.2%
0,Evening,398,170,42.7%


Products:
*   Digital Marketing accounts for the largest number of deals (1,944 deals), with a success rate of 24.3%—it's the most popular and consistently converting product.
*   UX/UI Design (22.6%) and Web Developer (24.1%) show similar success rates, but lower deal volume.
*   Find Yourself in IT has a very low number of deals.
*   Data Analytics has virtually no sales, indicating weak promotion or a new campaign.

Training Type:
*   Morning programs account for the majority of deals (2,837 deals), with a conversion rate of 23.2%.
*   However, Evening training demonstrates almost twice the conversion rate (42.7%), which may indicate more motivated or busy clients who choose evening formats.

Daytime programs generate the majority of sales, but evening programs show better conversion.

# Geographical analysis

## Analyze the geographic distribution of transactions by city

In [ ]:
from enum import unique
geo_df = deals[deals['Closing Date'].notna()][['Contact Name']].merge(
    contacts[['Id', 'City']],
    left_on='Contact Name',
    right_on='Id',
    how='left'
)
geo_df = (geo_df[geo_df['City'] != non].copy())
geo_stats = (
    geo_df.groupby(['City'], dropna=False)
    .agg(total_deals=('Id', 'count'))
    .reset_index()
)
geo_stats['Share (%)'] = (
    geo_stats['total_deals'] / geo_stats['total_deals'].sum() * 100
).round(2)
unique_city = (
    contacts[contacts['City'] != non][
        ['City', 'Country', 'State', 'Longitude', 'Latitude']
    ]
    .drop_duplicates(subset=['City'])
    .reset_index(drop=True)
)
geo_stats = geo_stats.merge(
    unique_city[['City', 'Country', 'State', 'Longitude', 'Latitude']],
    on='City',
    how='left'
)
top_cities = geo_stats.sort_values('total_deals', ascending=False)#.head(20)
display(top_cities[:15])

/tmp/ipython-input-1674340584.py:10: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,City,total_deals,Share (%),Country,State,Longitude,Latitude
83,Berlin,120,9.92,Germany,Berlin,13.404954,52.520007
496,München,39,3.22,Germany,Bavaria,11.581981,48.135125
280,Hamburg,32,2.64,Germany,Hamburg,9.98717,53.548828
408,Leipzig,20,1.65,Germany,Saxony,12.373075,51.339695
544,Nürnberg,20,1.65,Germany,Bavaria,11.074564,49.454288
214,Frankfurt,19,1.57,Germany,Hessen,8.682127,50.110922
170,Düsseldorf,17,1.40,Germany,North Rhine-Westphalia,6.782455,51.223041
162,Dresden,15,1.24,Germany,Saxony,13.737262,51.050409
389,Köln,15,1.24,Germany,North Rhine-Westphalia,6.960279,50.937531
161,Dortmund,14,1.16,Germany,North Rhine-Westphalia,7.465298,51.513587


The largest number of transactions were recorded in Germany's largest cities, reflecting their economic activity and concentration of potential clients.

*   Berlin leads the way, accounting for approximately 10% of all transactions, which is logical for the country's capital and largest business center.
*   Following are Munich (3.2%) and Hamburg (2.6%), also considered major technology and educational hubs.
*   Second-tier cities—Leipzig, Nuremberg, Frankfurt, and Düsseldorf—demonstrate a stable share (approximately 1.5% each).

The overall distribution of transactions confirms the concentration of activity in Germany's western and southern federal states, where the majority of clients and offices are concentrated.

In [ ]:
top_cities['Latitude'] = pd.to_numeric(top_cities['Latitude'], errors='coerce')
top_cities['Longitude'] = pd.to_numeric(top_cities['Longitude'], errors='coerce')
center_lat = top_cities['Latitude'].mean()
center_lon = top_cities['Longitude'].mean()
fig = px.scatter_mapbox(
    top_cities,
    lat="Latitude",
    lon="Longitude",
    hover_name="City",
    hover_data={"Country": True, "State": True, "total_deals": True, "Share (%)": True},
    color="total_deals",
    size="total_deals",
    color_continuous_scale=px.colors.sequential.Viridis,
    size_max=40,
    zoom=5,  # можно увеличить до 6–7 для большей детализации
    title="Geographical Distribution of Closed Deals"
)
fig.update_layout(
    mapbox_style="carto-positron",  # приятная нейтральная карта
    mapbox_center={"lat": center_lat, "lon": center_lon},
    height=700,
    template="plotly_white",
    coloraxis_colorbar=dict(title="Closed Deals")
)
fig.show()

## The influence of the level of knowledge of German on the success of transactions in different cities

In [ ]:
df = deals.merge(
    contacts[['Id', 'City', 'Level of Deutsch']],
    left_on='Contact Name',
    right_on='Id',
    how='left'
)
df['is_success'] = df['Stage'].eq('Payment Done')
lang_success = (
    df.groupby(['City', 'Level of Deutsch'], observed=True)
    .agg(
        total_deals=('Id_x', 'count'),
        successful_deals=('is_success', 'sum')
    )
    .reset_index()
)
lang_success['conversion_rate'] = (
    lang_success['successful_deals'] / lang_success['total_deals'] * 100
).round(1)
lang_success = lang_success[((lang_success['City'] != 'unknown') &
                                (lang_success['Level of Deutsch'] != 'unknown')
)]
top_cities = (
    lang_success.groupby('City')['total_deals']
    .sum()
    .index
)
lang_success = lang_success[lang_success['City'].isin(top_cities)]
df = deals.merge(
    contacts[['Id', 'City', 'Level of Deutsch']],
    left_on='Contact Name',
    right_on='Id',
    how='left'
)
df = df[df['Level of Deutsch'].notna()]
df = df[df['Level of Deutsch'] != 'unknown']
df = df[df['City'].notna()]
df = df[df['City'] != 'unknown']
df['is_success'] = df['Stage'].eq('Payment Done')
city_lang = (
    df.groupby(['City', 'Level of Deutsch'], observed=True)
    .agg(
        total_deals=('Id_x', 'count'),
        successful_deals=('is_success', 'sum')
    )
    .reset_index()
)

city_lang['conversion_rate'] = (
    city_lang['successful_deals'] / city_lang['total_deals'] * 100
).round(1)
city_mode = (
    city_lang.sort_values(['City', 'total_deals'], ascending=[True, False])
    .groupby('City')
    .first()
    .reset_index()
    [['City', 'Level of Deutsch']]
)
city_summary = (
    city_lang.groupby('City', observed=True)
    .agg(
        total_deals=('total_deals', 'sum'),
        successful=('successful_deals', 'sum')
    )
    .reset_index()
)
city_summary['conversion_rate'] = (
    city_summary['successful'] / city_summary['total_deals'] * 100
).round(1)
city_summary = city_summary.merge(city_mode, on='City', how='left')
city_summary = (city_summary.sort_values('total_deals', ascending=False)
                                                        .reset_index(drop=True))

print("City-level German proficiency and deal success summary:")
display(city_summary.head(15))


City-level German proficiency and deal success summary:


/tmp/ipython-input-3188537826.py:23: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

/tmp/ipython-input-3188537826.py:53: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,City,total_deals,successful,conversion_rate,Level of Deutsch
0,Berlin,129,41,31.8,B1
1,Hamburg,38,10,26.3,B1
2,Düsseldorf,34,5,14.7,B1
3,Nürnberg,32,7,21.9,B1
4,Leipzig,32,13,40.6,B1
5,München,26,10,38.5,B1
6,Dresden,19,7,36.8,B1
7,Duisburg,17,5,29.4,B1
8,Karlsruhe,15,3,20.0,B1
9,Köln,15,5,33.3,B1


Most deals are concluded in cities with a predominant German B1 level—this is evident in all 30 leading cities.

This indicates that clients with an intermediate level of German proficiency are most actively engaged in the learning and purchasing process.

Conversion rate:
*   the average conversion rate by city ranges from 20% to 40%, which is a stable figure
*   the cities with the highest conversion rates are Oberhausen (77.8%), Chemnitz (57.1%), Essen (50%), and Villingen-Schwenningen (44.4%)—indicating more motivated clients or effective local managers
*   in larger cities (Berlin, Hamburg, Munich), the conversion rate is below average (around 25–35%)—likely due to high competition and a higher number of disinterested leads

In [ ]:
df = deals[deals['Closing Date'].notna()].copy()
df = df.merge(
    contacts[['Id', 'City', 'Level of Deutsch', 'Longitude', 'Latitude']],
    left_on='Contact Name',
    right_on='Id',
    how='left'
)
df = df[
    (df['City'].notna()) &
    (df['City'] != 'unknown') &
    (df['Level of Deutsch'].notna()) &
    (df['Level of Deutsch'] != 'unknown')
].copy()
city_lang = (
    df.groupby('City', dropna=False)
    .agg(
        total_deals=('Id_x', 'count'),
        mode_level=('Level of Deutsch', lambda x: x.mode().iloc[0] if not x.mode().empty else 'unknown'),
        lon=('Longitude', 'first'),
        lat=('Latitude', 'first')
    )
    .reset_index()
)
city_lang['lat'] = pd.to_numeric(city_lang['lat'], errors='coerce')
city_lang['lon'] = pd.to_numeric(city_lang['lon'], errors='coerce')
city_lang = city_lang.dropna(subset=['lat', 'lon'])
color_order = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
color_map = {
    'A1': '#ff5959',
    'A2': '#ffad33',
    'B1': '#ffe066',
    'B2': '#5cd65c',
    'C1': '#4da6ff',
    'C2': '#b366ff'
}
center_lat = city_lang['lat'].mean()
center_lon = city_lang['lon'].mean()

fig = px.scatter_mapbox(
    city_lang,
    lat='lat',
    lon='lon',
    size='total_deals',
    color='mode_level',
    color_discrete_map=color_map,
    category_orders={'mode_level': color_order},
    hover_name='City',
    hover_data={
        'mode_level': True,
        'total_deals': True,
        'lat': False,
        'lon': False
    },
    title='Most Common German Language Level by City (Closed Deals)',
    zoom=5
)
fig.update_layout(
    mapbox_style='carto-positron',
    mapbox_center={'lat': center_lat, 'lon': center_lon},
    height=700,
    margin=dict(l=20, r=20, t=50, b=20),
    template='plotly_white'
)
fig.show()

/tmp/ipython-input-3725687041.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

